## Data sourcing

Source data from various source systems and ingest them using python code.

1. Parquet files
2. CSV files
3. APIs
4. RDBMS databases
5. HTML

In [3]:
!py -m pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for altair==4.2.2 from https://files.pythonhosted.org/packages/18/62/47452306e84d4d2e67f9c559380aeb230f5e6ca84fafb428dd36b96a99ba/altair-4.2.2-py3-none-any.whl.metadata
  Obtaining dependency information for annotated-types==0.6.0 from https://files.pythonhosted.org/packages/28/78/d31230046e58c207284c6b2c4e8d96e6d3cb4e52354721b944d3e1ee4aa5/annotated_types-0.6.0-py3-none-any.whl.metadata
  Obtaining dependency information for anyio==4.2.0 from https://files.pythonhosted.org/packages/bf/cd/d6d9bb1dadf73e7af02d18225cbd2c93f8552e13130484f1c8dcfece292b/anyio-4.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for argon2-cffi==23.1.0 from https://files.pythonhosted.org/packages/a4/6a/e8a041599e78b6b3752da48000b14c8d1e8a04ded09c88c714ba047f34f5/argon2_cffi-23.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for argon2-cffi-bindings==21.2.0 from https

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'D:\\Users\\leon.head\\AppData\\Roaming\\Python\\Python312\\site-packages\\~yzmq.libs\\msvcp140-587ef1e98d22156d27ef55ba57e44cb3.dll'
Check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# import modules
import certifi
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import urllib3
from urllib3 import request
import requests
from unicodedata import normalize

ModuleNotFoundError: No module named 'certifi'

### Sourcing Parquet data

Please visit the url https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [ ]:
# Read data from the Parquet file. We use pandas read_parquet method for ease and speed.
df_parquet = pd.read_parquet("yellow_tripdata_2022-01.parquet")
df_parquet.head()

### Sourcing CSV data 

Please visit the url https://data.cityofnewyork.us/resource/h9gi-nx95.csv?$limit=500


In [ ]:
# Read data from the CSV file. We use pandas read_csv method for ease and speed.
df_csv = pd.read_csv("h9gi-nx95.csv")
df_csv.head()

### Sourcing data from APIs

Please make sure to install the certifi library using - pipenv install certifi

In [4]:
# get api data from url
url = 'https://data.cityofnewyork.us/resource/h9gi-nx95.json?$limit=500'

# Check if API is available to retrive the data
apt_status = http.request('GET', url).status
print(apt_status)
if apt_status == 200:
    # Sometimes we get certificate error . We shoul never silence this error as this may cause a securirty threat.
    # Create a Pool manager that can be used to read the API response 
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
    data = json.loads(http.request('GET', url).data.decode('utf-8'))
    df_api = pd.json_normalize(data)
else:
    df_api = pd.Dataframe()
df_api.head(10)

NameError: name 'http' is not defined

### PISA API Testing

In [1]:
type = 'prod'

In [2]:
import pyspark,psycopg2
from pyspark.sql import SparkSession,SQLContext

spark = SparkSession.builder \
  .master("local[1]") \
  .appName("chapter6_schemas") \
  .config("spark.executor.memory", '3g') \
  .config("spark.executor.cores", '1') \
  .config("spark.cores.max", '1') \
  .config("spark.jars.packages","org.postgresql:postgresql:42.0.0") \
  .getOrCreate()

:: loading settings :: url = jar:file:/home/rstudio/workspaces/pisa2025-api-etl/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/rstudio/.ivy2/cache
The jars for the packages stored in: /home/rstudio/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-45020acc-786f-4265-9cf0-7df5761bd884;1.0
	confs: [default]
	found org.postgresql#postgresql;42.0.0 in central
:: resolution report :: resolve 175ms :: artifacts dl 2ms
	:: modules in use:
	org.postgresql#postgresql;42.0.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-45020acc-786f-4265-9cf0-7df5761bd884
	confs:

24/02/29 22:54:42 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
import sys
import requests
import json
import pandas as pd
import logging
from datetime import date

logging.captureWarnings(True)

##
##    function to obtain a new OAuth 2.0 token from the authentication server
##
def get_new_token(type: str):

    if(type=="staging"):
        auth_server_url = "https://auth-pisa-qa.staging.gcp-eu.taocloud.org/v1/oauth2/tokens"
        client_id = 'Q2caBAhwvNF8NgmrEPKrUw'
        client_secret = 'wHtKRe6eA7BWRBNIISiTQk3uM210OJdQ'
    elif(type=='prod'):
        auth_server_url = "https://auth-pisa-prod.prod.gcp-eu.taocloud.org/v1/oauth2/tokens"
        client_id = 'Amb9_PvLT4I2ui8flocJsA'
        client_secret = 'a8OIzcsHn0z8Z7mtYG9zfjXZotrIJQS1lNuEKEVSFHs'

    token_req_payload = {'grant_type': 'client_credentials'}

    token_response = requests.post(auth_server_url,
    data=token_req_payload, verify=False, allow_redirects=False,
    auth=(client_id, client_secret))
                
    if token_response.status_code !=200:
        print("Failed to obtain token from the OAuth 2.0 server", file=sys.stderr)
        sys.exit(1)

    print("Successfuly obtained a new token")
    tokens = json.loads(token_response.text)
    return tokens['access_token']

## 
## 	obtain a token before calling the API for the first time
##
token = get_new_token(type)


Successfuly obtained a new token


In [3]:
# API request function

def req_func(req_type,url,token,data=None):

##
##   call the API with the token
##
    
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ' + token
    }

    req_req = str.upper(req_type)

    response = requests.request(req_req, url, headers=headers, data=data)

    if	(response.status_code != 200):
        token = get_new_token(type)
        globals()['token'] = token
        headers = {
            'Content-Type': 'application/json',
            'Authorization': 'Bearer ' + token
        }
        response = requests.request(req_req, url, headers=headers, data=data)


    return response


In [4]:
def api_url(type: str, endpoint: str):
  if(type=='staging'):
    url = 'https://dynamic-query-api-pisa-qa.staging.gcp-eu.taocloud.org/api/v1/' + endpoint
  elif(type=='prod'):
    url = 'https://dynamic-query-api-pisa-prod.prod.gcp-eu.taocloud.org/api/v1/' + endpoint

  return url

In [5]:
api_url_health = api_url(type=type,endpoint='health')
df_health = req_func(req_type = "get", url=api_url_health,token=token)
df_health.text

'{"status":"ok"}'

In [10]:
api_url_entity_list = api_url(type=type,endpoint='entity/list')
response = req_func(req_type = "get", url=api_url_entity_list,token=token)
df_entity_list = pd.json_normalize(json.loads(response.text))
df_entity_list

,name,description
0,portalSessions,Holds information on portal sessions
1,portalUserGroups,Holds information on portal user groups
2,portalGroups,Holds information on portal groups
3,portalUserSessions,Holds information on portal user sessions
4,portalUser,Holds information on portal user
5,portalBattery,Holds information on portal battery
6,testRunnerDeliveries,Holds information on test runner deliveries
7,datastoreDeliveries,Holds information on datastore deliveries
8,datastoreDeliveryResults,Holds information on datastore delivery results
9,datastoreItemResults,Holds information on datastore item results


In [6]:
def json_schema_get(entity,token):
    api_entity_get = api_url(type=type,endpoint=('entity/%s' % entity))
    df_entity_schema = req_func(req_type = "get",url=api_entity_get,token=token)
    return df_entity_schema

In [18]:
import pandas as pd

entities = df_entity_list['name']

with pd.ExcelWriter('all_entities_spec_' + type.upper() + '_' + '.xlsx') as writer:
    for ent in entities:    
        resp = json_schema_get(entity=ent,token=token)
        df = pd.json_normalize(resp.json())
        df.to_excel(writer,sheet_name = ent, index = False)
        worksheet = writer.sheets[ent]  # pull worksheet object
        for idx, col in enumerate(df):  # loop through all columns
            series = df[col]
            max_len = max((
                series.astype(str).map(len).max(),  # len of largest item
                len(str(series.name))  # len of column name/header
                )) + 1  # adding a little extra space
            worksheet.set_column(idx, idx, max_len)  # set column width

In [7]:
def api_json_extract(response,entity,options=None):

  json_raw = response.json()
  if(options['pandas'] is True):
    df_raw = pd.json_normalize(json_raw,record_path=['data'],max_level = 0)
    # if(entity == 'datastoreDeliveryResults'):
    #   df_raw = df_raw.rename(columns={'rawItems':'raw_data'})
  else: df_raw = json_raw

  # if(options is not None):
  #   if (entity == 'datastoreDeliveryResults'):
  #     if('json_var' in options):
  #       json_var = options['json_var']
  #       tmp = df_raw.iloc[0]['data'][0][json_var]
  #       df = (
  #         tmp
  #         .join(pd.json_normalize(pd.json_normalize(tmp['values'])[0]))
  #         .drop(columns=['values'])
  #       )
  #       df['varType'] = df['type'].str.split(pat = "#").str[1]
  #     elif(('item_resp' in options) & (options['item_resp'] is True)):
  #       tmp = (
  #           pd.json_normalize(df_raw.iloc[0]['data'][0])
  #           .filter(regex = '^(?!metadata)(?!ltiParameters)(?!rawLtiParameters)(?!outcomes)')
  #         )
      
  #       idvars = tmp.columns[~tmp.columns.str.startswith('items.item-')]
  #       pivotvars = tmp.columns[tmp.columns.str.startswith('items.item-')]
        
  #       out = (
  #         pd.melt(
  #             tmp,
  #             id_vars = idvars,
  #             var_name = 'key',
  #             value_name = 'q_val',
  #             value_vars = pivotvars
  #         )
  #         .assign(
  #           key=lambda df: df.key.replace('items.item-','',regex=True)
  #         )
  #         .assign(
  #           q_num = lambda df: df.key.str.split('.',n=1).str[0],
  #           q_lab = lambda df: df.key.str.split('.',n=1).str[1],
  #         )
  #         .drop(
  #           ['key'],
  #           axis = 1
  #         )
  #       )
  #   else:
  #     out = (
  #       pd
  #       .json_normalize(df_raw.iloc[0]['data'][0])
  #     )
  # else:
  #   out = json_raw

  return df_raw

In [8]:
def payload_ent(entity: str, pageNumber=None, pageSize=None, filters=None, fields=None,flatResponse=None, sort = None, searchAfter = None):

  payload_dict = {}

  if(pageSize is not None):
    if(isinstance(pageSize,int)):
      payload_dict['pageSize'] = pageSize

  if(filters is not None):
    if(isinstance(filters,list)):
      payload_dict['filters'] = filters

  response = {}
  if(flatResponse is not None):
    response['flatResponse'] = False
  if(fields is not None):
    response['fields'] = fields

  if(len(response) > 0):
    payload_dict['response'] = response

  if(sort is not None):
    if(len(sort) > 0):
      payload_dict['sort'] = sort
  
  if(pageNumber is not None):
    payload_dict['pageNumber'] = pageNumber

  if(searchAfter is not None):
    payload_dict['searchAfter'] = searchAfter

  return json.dumps(payload_dict)

In [27]:
filter_dict = {}
# filter_dict['login'] = [
#   'TSC7011','TSC7012','TSC7013','TSC7014','TSC7015','TSC7016','TSC7017','TSC7018','TSC7019','TSC7020','TSC7021','TSC7022','TSC7041','TSC7042','TSC7043','TSC7044','TSC7045','TSC7046','TSC7047','TSC7048','TSC7049','TSC7050','TSC7051','TSC7071','TSC7072','TSC7073','TSC7074','TSC7075','TSC7076','TSC7077','TSC7078','TSC7079','TSC7081','TSC7101','TSC7102','TSC7103','TSC7104','TSC7105','TSC7106','TSC7107','TSC7108','TSC7109','TSC7110','TSC7111','TSC7112','TSC7113',
# ]
# filter_dict['deliveryId'] = [
#   "e9fc52080732",
#   "82c00e6ddbee",
#   "641b1e05580d",
#   "926ed4b705e4",
#   "eecc20e16409"
# ]
filter_dict['deliveryExecutionId'] = del_ids.tolist()
# filter_dict['login'] = [
#   '18289999011','18289999001','18289999014','18289999017','18289999044','18289999081','18289999002','18289999020','18289999035','18289999029','18289999083','18289999003','18289999021','18289999085','18289999087','18289999091','18289999007','18289999015','18289999068','18289999008','18289999093','18289999094','18289999095','18289999009','18289999010','18289999026','18289999096','18289999097','18289999099','18289999032','18289999041',
# ]

# filter_dict['testQtiId'] = [
#   'LDW1-LDW2',
#   'LDW2-LDW6',
#   'LDW3-LDW4',
#   'LDW4-LDW5',
#   'LDW5-LDW3',
#   'LDW6-LDW7',
#   'LDW7-LDW8',
#   'LDW8-LDW1',
#   'LDW8-LDW3',
#   'LDW1-LDW5',
#   'LDW2-LDW4',
#   'LDW3-LDW6',
#   'LDW4-LDW7',
#   'LDW5-LDW8',
#   'LDW6-LDW1',
#   'LDW7-LDW2'
# ]

filter_payload = [
  {"field": k, 'type':'terms', 'values': v} for k, v in filter_dict.items()
]
# deliveries = ["f8a87921b1e5"]
# deliveries = ["58e42b858a4b"]

In [10]:
if('login' in filter_dict):

  log = filter_dict['login']
  log_reverse = [x[::-1] for x in filter_dict['login']]
  pd.DataFrame(
    {
      "logins": log,
      "logins_reversed": log_reverse
    }
  ).to_excel('./data/ldw_logins.xlsx',index=False)

In [11]:
import datetime

epoch = datetime.datetime.utcfromtimestamp(0)
date_from = datetime.datetime(2024,3,26)
date_to = datetime.datetime(2024,4,14)
inc_date_to = True
inc_date_from = True

if(inc_date_to and inc_date_from):
  inclusive_term = 'both'
elif(inc_date_to and not inc_date_from):
  inclusive_term = 'right'
elif(inc_date_from and not inc_date_to):
  inclusive_term = 'left'

def unix_time_millis(dt):
    return int((dt - epoch).total_seconds() * 1000.0)

def unix_time_string(num: int):
  x = datetime.datetime.fromtimestamp(int(num)/1000).strftime("%Y-%m-%d %H:%M:%S")
  return x

def epoch_range(inc_date_to: bool, inclusive: str, date_from=None, date_to=None, periods = None):
    
  dates = pd.Series(pd.date_range(
    start = date_from,
    end = date_to,
    periods = periods,
    freq="1H",
    inclusive=inclusive
  )).apply(
    unix_time_millis
  )

  return dates

dates_all = epoch_range(inc_date_to=False, inclusive=inclusive_term, date_from = date_from, date_to = date_to, periods = None)

In [12]:
ent_cols = {}

ent_cols['datastoreDeliveryResults']=[
  "deliveryId",
  "login",
  "deliveryExecutionId",
  "submissionTime",
  "testQtiId",
  "testQtiTitle",
  "testQtiLabel",
  "last_update_date",
  "sessionStartTime",
  "sessionEndTime",
  "responses",
  "outcomes",
  "isDeleted",
  "rawItems"
]
ent_cols['datastoreDeliveries'] = [
  "deliveryId",
  "cutScoreRatio",
  "id",
  "items",
  "last_update_date",
  "messageId",
  "metadata",
  "metadataRaw",
  "outcomeDeclarations",
  "publicationTime",
  "qtiId",
  "qtiPackage",
  "qtiTitle",
  "structure",
  "tenantId",
  "testQtiId",
  "testQtiLabel",
  "testQtiTitle",
]
ent_cols['datastoreUIEvents'] = [
  "deliveryExecutionId",
  "batteryId",
  "itemId",
  "deliveryId",
  "domEventType",
  "tenantId",
  "responseId",
  "timestamp",
  "last_update_date",
  "_id",
  "metadata",
  "metadataRaw"
]

In [28]:
import math

entities = [
  # 'datastoreDeliveryResults',
  'datastoreUIEvents',
  # 'datastoreDeliveries'
]

resp_df = False

ent_df = {}
log_df = {}

for ent in entities:
  url = api_url(type=type,endpoint=('search/%s' % ent))

  if(ent=="datastoreUIEvents"):
    time_var = 'timestamp'
  else:
    time_var = 'last_update_date'

  fields = None

  sort = [
    {
      "type": "ASC",
      "field": time_var
    }
  ]

  filters_cnt=filter_payload + [{k: v for k, v in (('field', time_var), ('type', 'range'),('fromValue',unix_time_millis(date_from)),('toValue',unix_time_millis(date_to)),('toValueIncluded',False)) if v is not None}]
  
  payload_cnt = payload_ent(entity=ent, filters=filters_cnt, sort=sort, pageSize = 0)
  resp_cnt = req_func(req_type='post',url=url,token=token, data=payload_cnt)

  datas = []
  last_id = []
  log = []
  payloads = []
  dates = dates_all.to_list()
  for idx, x in enumerate(dates):

    from_val = unix_time_millis(date_from) if (idx is 0) else dates[idx-1]
      
    filters = filters_cnt + [{
      "field": time_var,
      "type": "range",
      "fromValue": from_val,
      "toValue": x,
      "toValueIncluded": False
    }]
    
    # if(idx != 0 and len(last_id) > 0):
    #   searchAfter = last_id
    #   payload = payload_ent(entity=ent, sort=sort, filters=filters, flatResponse=False,fields=None,searchAfter=searchAfter)
    # else:

    payload = payload_ent(entity=ent, sort=sort, filters=filters, flatResponse=False,fields=None)

    payloads.append(payload)
    resp = req_func(req_type='post',url=url,token=token, data=payload)
    resp_last_id = resp.json()['lastId']
    resp_total_results = resp.json()['totalResults']
    
    if(resp_total_results>0):
      last_id = resp_last_id
      log_resp = pd.DataFrame([last_id]).rename({0:'lastId_' + time_var,1:'lastId_id'},axis = 1)
      df = api_json_extract(resp, entity = ent, options = {'pandas': resp_df})
      log_resp['num'] = len(df['data'])
      log_resp[time_var] = int(x)
      datas.append(df)
      log.append(log_resp)
    else:
      last_id = last_id
  
  ent_df[ent] = datas.copy()
  if(len(log) > 0):
    log_df[ent] = pd.concat(log).reset_index(drop=True)

Successfuly obtained a new token


In [29]:
import copy
def json_concat(dat_list):
  json_obj = copy.deepcopy(dat_list[0])

  if(len(dat_list) > 1):
    for d in dat_list[1:]:
      json_obj['data'].extend(d['data'])

  json_obj['totalResults'] = len(json_obj['data'])

  return json_obj

def json_to_pd(json_raw):
  df_raw = pd.json_normalize(json_raw,record_path=['data'],max_level = 0)
  return df_raw

In [31]:
pd.set_option('display.width',200)
logs = log_df['datastoreUIEvents']
logs['index'] = logs[time_var].apply(lambda x: dates_all[dates_all==x].index[0])
logs['date_of_call']=logs[time_var].apply(lambda x: unix_time_string(x))
print(logs)
print('Total records in data file: ' + str(logs.num.sum()))
print('Total records according to API: ' + str(resp_cnt.json()['totalResults']))

     lastId_timestamp  lastId_id   num      timestamp  index         date_of_call
0       1711435163946   49162917   499  1711436400000      7  2024-03-26 18:00:00
1       1711438204578   49196661  5000  1711440000000      8  2024-03-26 19:00:00
2       1711440321893   30112525  5000  1711443600000      9  2024-03-26 20:00:00
3       1711443969835   31314951  5000  1711447200000     10  2024-03-26 21:00:00
4       1711447731859   34120285  5000  1711450800000     11  2024-03-26 22:00:00
..                ...        ...   ...            ...    ...                  ...
249     1712966301709   87881642  2108  1712966400000    432  2024-04-13 10:00:00
250     1712966590961   87882053   366  1712970000000    433  2024-04-13 11:00:00
251     1712991590397   87869995   474  1712991600000    439  2024-04-13 17:00:00
252     1712995166850   95470414  1470  1712995200000    440  2024-04-13 18:00:00
253     1712995668367   95479001   193  1712998800000    441  2024-04-13 19:00:00

[254 rows x 6 c

In [32]:
for k, v in ent_df.items():
  json_out = json_concat(v)
  filename = './data/' + k + '_' + type.upper() + '_' + date.today().strftime('%Y%m%d') + '.json'
  string = json.dumps(json_out)
  json_data = json.loads(
    string
      .replace("\\t", "\\\\t")
      .replace("\\n", "\\\\n")
  )
  with open(filename, 'w') as output_data:
    output_data.write(
      json.dumps(json_data, ensure_ascii=False)
    )

  df_raw = json_to_pd(json_data)
  globals()[k] = df_raw
  df_raw.to_excel('./data/' + k + '_' + type.upper() + '_' + date.today().strftime('%Y%m%d') + '.xlsx',index=False)

In [1]:
import psycopg2
import psycopg2.extras
from etl.create_table import *
from etl.psycopg2_database_helper import *

params = {
  'host': '10.3.5.186',
  'database': 'RawDB',
  'user': 'leon',
  'password': 'LsXBrXnf#rD8!F?T',
  'port': 5432
}

conn = get_postgres_connection(
  host=params['host'],
  database=params['database'],
  user = params['user'],
  password = params['password'],
  port = params['port']
)

cursor = conn.cursor()
conn.autocommit = True

In [ ]:
# sql = '''CREATE DATABASE "RawDB" WITH OWNER = postgres ENCODING = 'UTF8' LOCALE_PROVIDER = 'libc' CONNECTION LIMIT = -1 IS_TEMPLATE = False;'''

# # Executing the query 
# cursor.execute(sql) 
# sql = '''
#         CREATE TABLE delivery_results (
#             delivery_execution_id VARCHAR(100) PRIMARY KEY,
#             delivery_id character(12) NOT NULL,
#             is_deleted BOOL NOT NULL,
#             last_update_date BIGINT NOT NULL,
#             login VARCHAR(50) NOT NULL,
#             test_qti_id VARCHAR(255) NOT NULL,
#             test_qti_label VARCHAR(255) NOT NULL,
#             test_qti_title VARCHAR(255) NOT NULL,
#             raw_data TEXT NOT NULL
#         );
#         '''
# cursor.execute(sql) 

# sqlfile = open('./data/schema.sql','r')
# cursor.execute(sqlfile.read())

# close communication with the PostgreSQL database server
# cursor.close()
# commit the changes
# conn.commit()

In [5]:
import etl.postgresqlschemareader as pgsr

pgsr_tables = pgsr.get_tables(conn)
pgsr_cols = pgsr.get_columns(conn,'public','delivery_results')
pgsr_tree = pgsr.get_tree(conn)

pgsr.print_tree(pgsr_tree)
pgsr.print_columns(pgsr_cols)

oat.data_entries
 |-row_id (bigint)
 |-student (character varying)
 |-school (character varying)
 |-team (character varying)
 |-locale (character varying)
 |-template (character varying)
 |-metadata (jsonb)
 |-inserted_date (timestamp without time zone)
 |-last_update_date (timestamp without time zone)
oat.dedup_jobs
 |-row_id (bigint)
 |-dedup_table (character varying)
 |-status (integer)
 |-created_at (timestamp without time zone)
 |-num_records (integer)
 |-last_update_date (timestamp without time zone)
 |-last_update_timestamp (character varying)
 |-processed_at (timestamp without time zone)
oat.deliveries
 |-row_id (bigint)
 |-delivery_id (character varying)
 |-tenant_id (character varying)
 |-id (character varying)
 |-last_update_date (timestamp without time zone)
 |-test_qti_id (character varying)
 |-test_qti_label (character varying)
 |-test_qti_title (character varying)
 |-raw_data (jsonb)
oat.deliveries_old2
 |-row_id (bigint)
 |-delivery_id (character varying)
 |-tenant_id (

In [2]:
def sql_dat(dat):
  meta_cols = [
    # 'battery_id',
    'delivery_execution_id',
    'delivery_id',
    'isDeleted',
    'last_update_date',
    'login',
    'outcomes',
    'test_qti_id',
    'test_qti_label',
    'test_qti_title',
    'raw_data'
  ]

  dat_new = (
    dat
    .rename(
      columns={
        'batteryId':'battery_id',
        'deliveryExecutionId':'delivery_execution_id',
        'deliveryId':'delivery_id',
        'testQtiId':'test_qti_id',
        'testQtiLabel':'test_qti_label',
        'testQtiTitle':'test_qti_title',
        'rawItems':'raw_data'
      }
    )
    .filter(
      # regex = "|".join(meta_cols[0:-1]) + '|^items\.',
      regex = "|".join(meta_cols),
      axis=1
    )
  )

  return dat_new

In [63]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from psycopg2.extras import Json
from psycopg2.extensions import register_adapter

register_adapter(dict, Json)

all_deliveries_results = sql_dat(datastoreDeliveryResults)

all_deliveries_results['outcomes'] = all_deliveries_results['outcomes'].apply(lambda x: json.dumps(x))
all_deliveries_results['raw_data'] = all_deliveries_results['raw_data'].apply(lambda x: json.dumps(x))

df_json = spark.createDataFrame(all_deliveries_results) \
  .withColumnRenamed(
    'rawItems','raw_data'
  )

df_json.printSchema()

upsert_spark_df_to_postgres(
  dataframe_to_upsert=df_json,
  table_name='oat."deliveryResults"',
  table_unique_key=['deliveryExecutionId'],
  database_credentials=params
)

root
 |-- login: string (nullable = true)
 |-- test_qti_id: string (nullable = true)
 |-- test_qti_title: string (nullable = true)
 |-- delivery_id: string (nullable = true)
 |-- isDeleted: boolean (nullable = true)
 |-- outcomes: string (nullable = true)
 |-- last_update_date: long (nullable = true)
 |-- delivery_execution_id: string (nullable = true)
 |-- test_qti_label: string (nullable = true)
 |-- raw_data: string (nullable = true)




#################################################
 Total records loaded - 34
 Total records rejected - 34
#################################################

 Started Printing Error Messages ....
['column "test_qti_id" of relation "deliveryResults" does not exist\nLINE 1:  INSERT INTO oat."deliveryResults" (login, test_qti_id, test...\n                                                   ^\n', 'column "test_qti_id" of relation "deliveryResults" does not exist\nLINE 1:  INSERT INTO oat."deliveryResults" (login, test_qti_id, test...\n                                                   ^\n', 'column "test_qti_id" of relation "deliveryResults" does not exist\nLINE 1:  INSERT INTO oat."deliveryResults" (login, test_qti_id, test...\n                                                   ^\n', 'column "test_qti_id" of relation "deliveryResults" does not exist\nLINE 1:  INSERT INTO oat."deliveryResults" (login, test_qti_id, test...\n                                                   ^\n', 'column "te

In [229]:
import glob, re
from pyspark.sql.functions import *
from pyspark.sql.types import *

json_files = glob.glob("./data/datastoreDeliveryResults.json")
# json_files = [x for x in json_files if 'db328a265d14' in x]

for f in json_files:
  df_json = spark.read.option("multiline","true") \
    .json(f) \
    .select(
      '*',
      explode('data').alias("dataExplode")
    ) \
    .select("dataExplode.*") \
    .select(
      '*',
      explode('raw_data').alias("raw_dataExplode")
    ) \
    .select("raw_dataExplode.*") \
    .withColumn(
      'raw_data',
      to_json('raw_data')
    )
      # .selectExpr(
    #   'deliveryExecutionId as delivery_execution_id',
    #   'deliveryId as delivery_id',
    #   'isDeleted as is_deleted',
    #   'last_update_date',
    #   'login',
    #   'testQtiid as test_qti_id',
    #   'testQtiLabel as test_qti_label',
    #   'testQtiTitle as test_qti_title',
    #   'items as raw_data'
    # ) \
    
  
  df_json.show()
  df_json.printSchema()

  # upsert_spark_df_to_postgres(
  #   dataframe_to_upsert=df_json,
  #   table_name='delivery_results',
  #   table_unique_key=['delivery_execution_id'],
  #   database_credentials=params
  # )

In [2]:
fetch_query = f"""
    SELECT
        battery_id, delivery_execution_id, delivery_id, last_update_date, login,
        test_qti_id, test_qti_label, test_qti_title, raw_data
    FROM oat.delivery_results
    WHERE
        raw_data->'metadata'->>'PISA25 Domains' = 'https://www.oecd.org/FLA'
        AND (LENGTH(login) = 11 OR LENGTH(login) = 12)
        AND (login LIKE '1%' OR login LIKE '2%' OR login LIKE '5%' OR login LIKE 'd%')
        AND (login NOT LIKE '____9999%')
        AND (login NOT LIKE '____A9999%')
        AND (login NOT LIKE '%demo%')
    LIMIT 10000
    """

In [7]:
fetch_query = f"""
    SELECT
        battery_id, delivery_execution_id, delivery_id, last_update_date, login,
        test_qti_id, test_qti_label, test_qti_title, raw_data
    FROM oat.delivery_results
    WHERE
        regexp_like(test_qti_id,'^S')
        AND NOT regexp_like(test_qti_label, 'delivery|questionnaire|student')
        AND (LENGTH(login) = 11 OR LENGTH(login) = 12)
        AND (login LIKE '1%' OR login LIKE '2%' OR login LIKE '5%' OR login LIKE 'd%')
        AND (login NOT LIKE '____9999%')
        AND (login NOT LIKE '____A9999%')
        AND (login NOT LIKE '%demo%')
    """

In [4]:
import pandas as pd
import numpy as np
import glob
import re
from openpyxl import load_workbook

path = glob.glob("./data/*FLA.xlsx")

r_cols_sp = pd.DataFrame(
    {
        "old": [
            'Unique Item ID (TAO)',
            'Item Format',
            'Task topic'
        ],
        "new": [
            'qtiLabel',
            'item_format',
            'unit_name'
        ]
    }
)

r_cols = pd.DataFrame(
    {
        "old": [
            'Unit/task ID (TAO)',
            'Unique Item ID (TAO)',
            'Item Format',
            'Unit/task Title',
        ],
        "new": [
            'qtiLabel',
            'resp_cat',
            'item_format',
            'unit_name',
        ]
    }
)

cbk_l = []
wb = load_workbook(filename=path[0])
for sheet_names in wb.sheetnames:
    sheet = wb[sheet_names]
    df_name = sheet_names.split()[0]
    dat = pd.read_excel(path[0],sheet_name = sheet_names)
    
    if(df_name == 'Speaking'):
        dict_rename = dict(zip(r_cols_sp["old"], r_cols_sp["new"]))
    else:
        dict_rename = dict(zip(r_cols["old"], r_cols["new"]))
        
    dat = dat.rename(columns=lambda x: x.strip()).rename(columns=dict_rename)
    dat['domain'] = f"FLA-{df_name[0]}"
    if(df_name.lower() == 'speaking'):
        dat['resp_cat'] = 'RESPONSE'
    new_cols = list(r_cols['new'].unique()) + ['domain']
    dat = dat.loc[:,new_cols]
    dat['qtiLabel'] = dat['qtiLabel'].str.replace('\xa0', '')
    
    md_path = glob.glob(f"./data/metadata/{df_name.lower()}/*.csv")
    md_l = []
    for file in md_path:
        # md_file = pd.read_csv(file).loc[:,['label','responseIdentifier','BR_identifier']]
        md_file = pd.read_csv(file)
        md_file.rename(
            columns = {
                md_file.columns[0]: "qtiLabel",
                md_file.columns[3]: "resp_cat",
                md_file.columns[4]: "key"
            },
            inplace=True
        )

        id_cols = ['qtiLabel','resp_cat','key']
        resp_cols = [col for col in md_file if col.startswith('choice_identifier')]
        all_cols = [*id_cols,*resp_cols]
        md_long = pd.melt(
            md_file.loc[:,all_cols],
            id_vars = id_cols,
            value_vars = resp_cols,
            var_name = "col",
            value_name = "val"
        ).sort_values(
            ['qtiLabel','resp_cat','col']
        )

        md_long['col'] = md_long['col'].apply(
            lambda x: str(re.sub("choice_identifier_","",x))
        ).apply(
            lambda x: re.sub("7","A",x)
        ).apply(
            lambda x: re.sub("8","B",x)
        )

        md_wide = pd.pivot(
            md_long,
            index = ['qtiLabel','resp_cat','key'],
            columns = "col",
            values = "val"
        ).reset_index()

        # md_l.append(md_file.loc[:,all_cols])
        md_l.append(md_wide)
        
    md_resp = pd.concat(md_l)
    print(md_resp.shape)
    dat = pd.merge(dat,md_resp,how = 'left',on = ['qtiLabel','resp_cat'])
    cbk_l.append(dat)
    
cbk = pd.concat(cbk_l,ignore_index = True).dropna(subset = ['qtiLabel']).drop_duplicates()
cbk['subtask'] = cbk['resp_cat'].apply(lambda x: "RESPONSE_0" if x=="RESPONSE" else x).apply(lambda x: re.sub("RESPONSE_","",x))
cbk = cbk.sort_values(['qtiLabel','subtask'])
cbk['subtask'] = cbk.groupby('qtiLabel').cumcount() + 1
cbk['subtask'] = cbk['subtask'].apply(lambda x: str(x).zfill(2))
cbk['qtiLabel_cbk'] = np.where(cbk['domain'] == 'FLA-S',cbk['qtiLabel'],cbk['qtiLabel'] + cbk['subtask'])
cbk.head()

(159, 11)
(74, 7)
(80, 11)


TypeError: expected string or bytes-like object, got 'float'

In [103]:
import pandas as pd
import numpy as np
import glob
import re
from openpyxl import load_workbook

path = [f for f in glob.glob("./data/*.xlsx") if "codebook_analysis" in f and "FLA" in f]

wb = load_workbook(filename=path[0])
sheet_cq = [f for f in wb.sheetnames if "_CQ" in f]

r_cols = pd.DataFrame(
    {
        "old": [
            'Unique item ID',
            'Item type',
            'Key'
        ],
        "new": [
            'qtiLabel',
            'item_type',
            'key'
        ]
    }
)

cbk_l = []
wb = load_workbook(filename=path[0])
for sheet_names in sheet_cq[0:2]:

    df_name = sheet_names.split("_")[1]
    print(sheet_names)
    dat = pd.read_excel(path[0],sheet_name = sheet_names)
    dat = dat.loc[dat['Stimulus']==0,]
    dat['Unique item ID'] = dat['Unique item ID'].apply(lambda x: re.sub("FLA25","FLA",x))
    dat['domain'] = f"FLA-{df_name[0]}"

    dat = dat.rename(columns=lambda x: x.strip()).rename(columns=lambda x: x.replace('\xa0', '')).rename(columns=dict(zip(r_cols["old"], r_cols["new"])))
    new_cols = list(r_cols['new'].unique()) + ['domain','key']
    dat = dat.loc[:,new_cols]
    dat = dat.loc[:,'qtiLabel':'domain'].drop_duplicates()
    dat = dat.set_axis(['qtiLabel',"item_type",'cq_key','cq_key2','domain'],axis = 1)
    dat['qtiLabel'] = dat['qtiLabel'].str.replace('\xa0', '')
    dat.to_csv('./data/dat.csv')

    md_path = glob.glob(f"./data/metadata/{df_name.lower()}/*.csv")
    md_l = []
    for file in md_path:
        csv_file = pd.read_csv(file)
        csv_file.rename(
            columns = {
                csv_file.columns[0]: "unit_id",
                csv_file.columns[3]: "resp_cat",
                csv_file.columns[4]: "db_key"
            },
            inplace=True
        )
        csv_file = csv_file.dropna(subset = ['db_key'])

        if(len(csv_file.db_key)==0):
            continue

        if(csv_file.db_key.str.contains("gap").any()):
            dat_list = []
            for idrow,row in csv_file.iterrows():
                key_list = [re.sub("\\sgap_[0-9]","",x) for x in row['db_key'].split("|")]
                row_list = []
                for idx,x in enumerate(key_list):
                    row_df = pd.DataFrame(row).transpose()
                    resp_cols = [col for col in row_df if col.startswith('choice_identifier')]
                    dat_r = pd.DataFrame(
                        {
                            'unit_id': [row['unit_id']],
                            'resp_cat': ['RESPONSE_' + str(idx)],
                            'db_key': [x]
                        }
                    )
                    dat_r['resp_cat'] = dat_r['resp_cat'].apply(lambda x: re.sub("RESPONSE_0","RESPONSE",x))
                    dat_final = pd.concat(
                        [
                            dat_r.reset_index(drop = True),
                            row_df.loc[:,resp_cols].reset_index(drop = True)
                        ],
                        axis = 1
                    )
                    row_list.append(dat_final)

                dat_row = pd.concat(row_list,axis = 0)
                dat_list.append(dat_row)

            md_file = pd.concat(dat_list,axis = 0)
        else:
            md_file = csv_file

        id_cols = ['unit_id','resp_cat','db_key']
        resp_cols = [col for col in md_file if col.startswith('choice_identifier')]
        all_cols = [*id_cols,*resp_cols]
        md_long = pd.melt(
            md_file.loc[:,all_cols],
            id_vars = id_cols,
            value_vars = resp_cols,
            var_name = "col",
            value_name = "val"
        ).sort_values(
            ['unit_id','resp_cat','col']
        )

        md_long['col'] = md_long['col'].apply(
            lambda x: str(re.sub("choice_identifier_","",x))
        ).apply(
            lambda x: re.sub("7","A",x)
        ).apply(
            lambda x: re.sub("8","B",x)
        )

        md_wide = pd.pivot(
            md_long,
            index = ['unit_id','resp_cat','db_key'],
            columns = "col",
            values = "val"
        ).reset_index()

        md_l.append(md_wide)
        
    md_resp = pd.concat(md_l).dropna(subset=['unit_id','resp_cat'])
    md_resp['subtask'] = md_resp['resp_cat'].apply(lambda x: "RESPONSE_0" if x=="RESPONSE" else x).apply(lambda x: re.sub("RESPONSE_","",x))
    md_resp = md_resp.sort_values(['unit_id','subtask'])
    md_resp['subtask'] = md_resp.groupby('unit_id').cumcount() + 1
    md_resp['subtask'] = md_resp['subtask'].apply(lambda x: str(x).zfill(2))
    md_resp['qtiLabel'] = np.where(df_name[0] == 'S',md_resp['unit_id'],md_resp['unit_id'] + md_resp['subtask'])
    md_resp.to_csv('./data/md_resp.csv')
    print(md_resp.shape)
    dat = pd.merge(dat,md_resp.drop_duplicates(),how = 'left',on = ['qtiLabel'])
    cbk_l.append(dat)

cbk = pd.concat(cbk_l,ignore_index = True).drop_duplicates()

FLA_Reading_CQ
(180, 13)
col        unit_id    resp_cat    db_key         1         2         3  \
0    FLAR2IMCB2001    RESPONSE  choice_4  choice_1  choice_2  choice_3   
1    FLAR2IMCB2001  RESPONSE_1  choice_7  choice_5  choice_6  choice_7   
2    FLAR2IMCB2002    RESPONSE  choice_3  choice_1  choice_2  choice_3   
3    FLAR2IMCB2002  RESPONSE_1  choice_8  choice_5  choice_6  choice_7   
4    FLAR2IMCB2003    RESPONSE  choice_2  choice_1  choice_2  choice_3   
..             ...         ...       ...       ...       ...       ...   
10     FLARMSA1004    RESPONSE  choice_1  choice_5  choice_2  choice_1   
11     FLARMSA1004  RESPONSE_1  choice_3  choice_5  choice_2  choice_1   
12     FLARMSA1004  RESPONSE_2  choice_4  choice_5  choice_2  choice_1   
13     FLARMSA1004  RESPONSE_3  choice_6  choice_5  choice_2  choice_1   
14     FLARMSA1004  RESPONSE_4  choice_7  choice_5  choice_2  choice_1   

col         4         5         6         A    B subtask         qtiLabel  
0    choic

In [3]:
import pandas as pd
import pandas.io.sql as sqlio

df = sqlio.read_sql_query(fetch_query,conn)

df.head()

D:\Users\leon.head\AppData\Local\Temp\1\ipykernel_12428\1585037678.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = sqlio.read_sql_query(fetch_query,conn)


,battery_id,delivery_execution_id,delivery_id,last_update_date,login,test_qti_id,test_qti_label,test_qti_title,raw_data
0,None,46010004361#e5dbb6b33a06#628d87cd7a9548677796e...,e5dbb6b33a06,2024-03-10 07:01:17.034,16340001064,FLA-R-L1-FLA-R-H1-FLA-R-M5,FLA-R-L1-FLA-R-H1-FLA-R-M5 (en-ZZ),FLA-R-L1-FLA-R-H1-FLA-R-M5,{'_id': 'NDYwMTAwMDQzNjEjZTVkYmI2YjMzYTA2IzYyO...
1,None,46010004361#49d03a0434e8#628d87cd7a9548677796e...,49d03a0434e8,2024-03-10 07:04:43.043,16340001064,FLA-L-6,FLA-L-6 (en-ZZ),FLA-L-6,{'_id': 'NDYwMTAwMDQzNjEjNDlkMDNhMDQzNGU4IzYyO...
2,None,22010004361#c8a5e9542569#f70deb969efebf5c8d8ee...,c8a5e9542569,2024-03-10 07:05:07.481,16340001022,FLA-R-L3-FLA-R-M3-FLA-R-H3,FLA-R-L3-FLA-R-M3-FLA-R-H3 (en-ZZ),FLA-R-L3-FLA-R-M3-FLA-R-H3,{'_id': 'MjIwMTAwMDQzNjEjYzhhNWU5NTQyNTY5I2Y3M...
3,None,46010004361#c558df35e879#628d87cd7a9548677796e...,c558df35e879,2024-03-10 07:06:36.868,16340001064,FLA-S-2,FLA-S-2 (en-ZZ),FLA-S-2,{'_id': 'NDYwMTAwMDQzNjEjYzU1OGRmMzVlODc5IzYyO...
4,None,67010004361#f6df97ec74af#eab24fb48466d9403ae65...,f6df97ec74af,2024-03-10 07:08:14.611,16340001076,FLA-R-M4-FLA-R-H1-FLA-R-L3,FLA-R-M4-FLA-R-H1-FLA-R-L3 (en-ZZ),FLA-R-M4-FLA-R-H1-FLA-R-L3,{'_id': 'NjcwMTAwMDQzNjEjZjZkZjk3ZWM3NGFmI2VhY...


In [4]:
nc_dat = pd.read_excel('./data/NCcodes.xlsx').astype({'isocntcd': str})

json_dat = pd.json_normalize(df['raw_data'],max_level = 0)

tmp = pd.concat([df.drop(['raw_data'],axis = 1).assign(isocntcd=lambda x: x['login'].str.slice(1,4)),json_dat.drop(['login','last_update_date'],axis = 1)],axis = 1)

df1 = pd.merge(
    tmp,
    nc_dat.loc[:,['isocntcd','isoalpha3','isoname']],
    how = 'left',
    on = 'isocntcd'
)

df1.head()

,battery_id,delivery_execution_id,delivery_id,last_update_date,login,test_qti_id,test_qti_label,test_qti_title,isocntcd,_id,...,testQtiTitle,ltiParameters,sessionEndTime,submissionTime,publicationTime,rawLtiParameters,sessionStartTime,deliveryExecutionId,isoalpha3,isoname
0,None,46010004361#e5dbb6b33a06#628d87cd7a9548677796e...,e5dbb6b33a06,2024-03-10 07:01:17.034,16340001064,FLA-R-L1-FLA-R-H1-FLA-R-M5,FLA-R-L1-FLA-R-H1-FLA-R-M5 (en-ZZ),FLA-R-L1-FLA-R-H1-FLA-R-M5,634,NDYwMTAwMDQzNjEjZTVkYmI2YjMzYTA2IzYyOGQ4N2NkN2...,...,FLA-R-L1-FLA-R-H1-FLA-R-M5,"{'aud': 'portal-client-id-18', 'iss': 'https:/...",1710054074000,1710054074000,1709200085000,"{'aud': 'portal-client-id-18', 'iss': 'https:/...",1710053490000,46010004361#e5dbb6b33a06#628d87cd7a9548677796e...,QAT,Qatar
1,None,46010004361#49d03a0434e8#628d87cd7a9548677796e...,49d03a0434e8,2024-03-10 07:04:43.043,16340001064,FLA-L-6,FLA-L-6 (en-ZZ),FLA-L-6,634,NDYwMTAwMDQzNjEjNDlkMDNhMDQzNGU4IzYyOGQ4N2NkN2...,...,FLA-L-6,"{'aud': 'portal-client-id-18', 'iss': 'https:/...",1710054280000,1710054280000,1709200096000,"{'aud': 'portal-client-id-18', 'iss': 'https:/...",1710054152000,46010004361#49d03a0434e8#628d87cd7a9548677796e...,QAT,Qatar
2,None,22010004361#c8a5e9542569#f70deb969efebf5c8d8ee...,c8a5e9542569,2024-03-10 07:05:07.481,16340001022,FLA-R-L3-FLA-R-M3-FLA-R-H3,FLA-R-L3-FLA-R-M3-FLA-R-H3 (en-ZZ),FLA-R-L3-FLA-R-M3-FLA-R-H3,634,MjIwMTAwMDQzNjEjYzhhNWU5NTQyNTY5I2Y3MGRlYjk2OW...,...,FLA-R-L3-FLA-R-M3-FLA-R-H3,"{'aud': 'portal-client-id-18', 'iss': 'https:/...",1710054305000,1710054305000,1709200080000,"{'aud': 'portal-client-id-18', 'iss': 'https:/...",1710053927000,22010004361#c8a5e9542569#f70deb969efebf5c8d8ee...,QAT,Qatar
3,None,46010004361#c558df35e879#628d87cd7a9548677796e...,c558df35e879,2024-03-10 07:06:36.868,16340001064,FLA-S-2,FLA-S-2 (en-ZZ),FLA-S-2,634,NDYwMTAwMDQzNjEjYzU1OGRmMzVlODc5IzYyOGQ4N2NkN2...,...,FLA-S-2,"{'aud': 'portal-client-id-18', 'iss': 'https:/...",1710054394000,1710054394000,1709200143000,"{'aud': 'portal-client-id-18', 'iss': 'https:/...",1710054332000,46010004361#c558df35e879#628d87cd7a9548677796e...,QAT,Qatar
4,None,67010004361#f6df97ec74af#eab24fb48466d9403ae65...,f6df97ec74af,2024-03-10 07:08:14.611,16340001076,FLA-R-M4-FLA-R-H1-FLA-R-L3,FLA-R-M4-FLA-R-H1-FLA-R-L3 (en-ZZ),FLA-R-M4-FLA-R-H1-FLA-R-L3,634,NjcwMTAwMDQzNjEjZjZkZjk3ZWM3NGFmI2VhYjI0ZmI0OD...,...,FLA-R-M4-FLA-R-H1-FLA-R-L3,"{'aud': 'portal-client-id-18', 'iss': 'https:/...",1710054492000,1710054492000,1709200121000,"{'aud': 'portal-client-id-18', 'iss': 'https:/...",1710053043000,67010004361#f6df97ec74af#eab24fb48466d9403ae65...,QAT,Qatar


In [5]:
df2 = pd.concat([df1.drop(['items'],axis = 1),df1['items'].apply(pd.Series)],axis = 1)
all_vars = set(df1.columns) - set(['items'])

id_vars = [
    'isocntcd',
    'isoalpha3',
    'isoname',
    'login',
    'last_update_date',
    'test_qti_id',
    'sessionStartTime',
    'sessionEndTime'
]

value_vars = df2.drop(columns = all_vars,axis = 1).columns

df3 = pd.melt(df2,id_vars = id_vars,value_vars=value_vars,var_name = 'qtiLabel',value_name = 'value').dropna()
df3.head()

,isocntcd,isoalpha3,isoname,login,last_update_date,test_qti_id,sessionStartTime,sessionEndTime,qtiLabel,value
0,634,QAT,Qatar,16340001064,2024-03-10 07:01:17.034,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710053490000,1710054074000,cluster1-FLAR22-item-1,"{'qtiId': 'FLARDPGPREA1001', 'score': '1', 'du..."
110,634,QAT,Qatar,16340021007,2024-03-11 08:41:51.535,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710143231000,1710146509000,cluster1-FLAR22-item-1,"{'qtiId': 'FLARDPGPREA1001', 'score': '1', 'du..."
200,191,HRV,Croatia,11910095034,2024-03-12 23:53:40.479,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710227979000,1710229646000,cluster1-FLAR22-item-1,"{'qtiId': 'FLARDPGPREA1001', 'score': '1', 'du..."
255,634,QAT,Qatar,16340048102,2024-03-13 20:46:10.313,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710225990000,1710227858000,cluster1-FLAR22-item-1,"{'qtiId': 'FLARDPGPREA1001', 'score': '1', 'du..."
272,634,QAT,Qatar,16340050014,2024-03-13 20:46:25.613,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710229495000,1710230708000,cluster1-FLAR22-item-1,"{'qtiId': 'FLARDPGPREA1001', 'score': '1', 'du..."


In [6]:
df4 = df3.join(pd.json_normalize(df3['value'],max_level = 0).drop(['qtiLabel'],axis = 'columns').set_index(df3.index)).drop('value',axis = 'columns')
df4.head()

,isocntcd,isoalpha3,isoname,login,last_update_date,test_qti_id,sessionStartTime,sessionEndTime,qtiLabel,qtiId,...,maxScore,outcomes,qtiTitle,responses,itemEndTime,numAttempts,itemStartTime,statusCorrect,submissionTime,completionStatus
0,634,QAT,Qatar,16340001064,2024-03-10 07:01:17.034,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710053490000,1710054074000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1,"{'SCORE': '1', 'MAXSCORE': '1', 'completionSta...",FLARDPGPREA1001,"{'RESPONSE': {'value': 'A_1', 'correct': True}}",1.710054e+12,1,1.710053e+12,correct,1710054074000,completed
110,634,QAT,Qatar,16340021007,2024-03-11 08:41:51.535,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710143231000,1710146509000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1,"{'SCORE': '1', 'MAXSCORE': '1', 'completionSta...",FLARDPGPREA1001,"{'RESPONSE': {'value': 'A_1', 'correct': True}}",1.710143e+12,1,1.710143e+12,correct,1710146509000,completed
200,191,HRV,Croatia,11910095034,2024-03-12 23:53:40.479,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710227979000,1710229646000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1,"{'SCORE': '1', 'MAXSCORE': '1', 'completionSta...",FLARDPGPREA1001,"{'RESPONSE': {'value': 'A_1', 'correct': True}}",1.710228e+12,1,1.710228e+12,correct,1710229646000,completed
255,634,QAT,Qatar,16340048102,2024-03-13 20:46:10.313,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710225990000,1710227858000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1,"{'SCORE': '1', 'MAXSCORE': '1', 'completionSta...",FLARDPGPREA1001,"{'RESPONSE': {'value': 'A_1', 'correct': True}}",1.710226e+12,1,1.710226e+12,correct,1710227858000,completed
272,634,QAT,Qatar,16340050014,2024-03-13 20:46:25.613,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710229495000,1710230708000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1,"{'SCORE': '1', 'MAXSCORE': '1', 'completionSta...",FLARDPGPREA1001,"{'RESPONSE': {'value': 'A_1', 'correct': True}}",1.710230e+12,1,1.710229e+12,correct,1710230708000,completed


In [7]:
df_cnt = (
    df4[['login','qtiLabel']]
    .drop_duplicates()
    .groupby(by = ['login','qtiLabel'])
    .size()
    .to_frame(name = 'count')
    .reset_index()
    .assign(dup_std_item=lambda x: x['count'].apply(lambda y: 'duplicate' if y > 1 else ''))
    .assign(stim_item=lambda x: x['qtiLabel'].apply(lambda y: 'stim' if 'Q00' in y else ''))
)

df4 = df4.merge(
    df_cnt,
    on = ['login','qtiLabel']
)

In [8]:
df4['key_len'] = df4.apply(lambda x: len(x['qtiLabel']) > 3,axis = 1)

# resp_dat = df4[df4['key_len'] == True]
resp_dat = df4.loc[(df4['qtiId'] == 'FLARMMB2001')]
d = resp_dat[['qtiId','responses','score']].iloc[0]
print('qtiId: ' + d['qtiId'])
print('score: ' + str(d['score']))
d['responses']

qtiId: FLARMMB2001
score: 0


{'RESPONSE_1': {'value': None, 'correct': None}}

In [11]:
resp_dat[['qtiId','responses','score']].to_excel('./data/FLARMMB2001_20240517.xlsx')

In [134]:
invalid_cell = [(isinstance(x,list)) & (len(x)==0) for x in df4['responses']]

df4.loc[invalid_cell,['responses']] = {}
df4.head()

,isocntcd,isoalpha3,isoname,login,last_update_date,test_qti_id,sessionStartTime,sessionEndTime,qtiLabel,qtiId,...,numAttempts,itemStartTime,statusCorrect,submissionTime,completionStatus,state,count,dup_std_item,stim_item,key_len
0,634,QAT,Qatar,16340001064,2024-03-10 07:01:17.034,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710053490000,1710054074000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1,1.710053e+12,correct,1710054074000,completed,NaN,1,,,True
1,634,QAT,Qatar,16340021007,2024-03-11 08:41:51.535,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710143231000,1710146509000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1,1.710143e+12,correct,1710146509000,completed,NaN,1,,,True
2,191,HRV,Croatia,11910095034,2024-03-12 23:53:40.479,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710227979000,1710229646000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1,1.710228e+12,correct,1710229646000,completed,NaN,1,,,True
3,634,QAT,Qatar,16340048102,2024-03-13 20:46:10.313,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710225990000,1710227858000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1,1.710226e+12,correct,1710227858000,completed,NaN,1,,,True
4,634,QAT,Qatar,16340050014,2024-03-13 20:46:25.613,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710229495000,1710230708000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1,1.710229e+12,correct,1710230708000,completed,NaN,1,,,True


In [135]:
# import flatdict
# exp = df4['responses'].apply(flatdict.FlatDict).apply(pd.Series)

df5 = df4.join(pd.json_normalize(df4['responses']).set_index(df4.index))
df5.head()

,isocntcd,isoalpha3,isoname,login,last_update_date,test_qti_id,sessionStartTime,sessionEndTime,qtiLabel,qtiId,...,RESPONSE_6.value,RESPONSE_6.correct,RESPONSE_8.value,RESPONSE_8.correct,RESPONSE_9.value,RESPONSE_9.correct,RESPONSE_10.value,RESPONSE_10.correct,RESPONSE_7.value,RESPONSE_7.correct
0,634,QAT,Qatar,16340001064,2024-03-10 07:01:17.034,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710053490000,1710054074000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,634,QAT,Qatar,16340021007,2024-03-11 08:41:51.535,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710143231000,1710146509000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,191,HRV,Croatia,11910095034,2024-03-12 23:53:40.479,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710227979000,1710229646000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,634,QAT,Qatar,16340048102,2024-03-13 20:46:10.313,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710225990000,1710227858000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,634,QAT,Qatar,16340050014,2024-03-13 20:46:25.613,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710229495000,1710230708000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [136]:
import re

all_vars = set(df4.columns) - set(['responses'])

id_vars = ['isocntcd', 'isoalpha3', 'isoname', 'login', 'last_update_date',
       'test_qti_id', 'sessionStartTime', 'sessionEndTime', 'qtiLabel',
       'qtiId', 'score', 'duration', 'maxScore', 'qtiTitle',
       'itemEndTime', 'numAttempts', 'itemStartTime',
       'statusCorrect', 'submissionTime', 'completionStatus', 'count',
       'dup_std_item', 'stim_item']

value_vars = df5.drop(columns = all_vars,axis = 1).drop('responses',axis = 'columns').columns

df6 = pd.melt(df5,id_vars = id_vars,value_vars=value_vars,var_name = 'resp',value_name = 'resp_value').dropna()
df6[['resp_cat','resp_type']] = df6['resp'].str.split('.',expand = True)
df6['resp_cat'] = df6['resp_cat'].apply(lambda x: re.sub('.*(?=RESPONSE)','',x))
df6 = df6.drop(columns = ['resp'])
df6.head()

,isocntcd,isoalpha3,isoname,login,last_update_date,test_qti_id,sessionStartTime,sessionEndTime,qtiLabel,qtiId,...,itemStartTime,statusCorrect,submissionTime,completionStatus,count,dup_std_item,stim_item,resp_value,resp_cat,resp_type
0,634,QAT,Qatar,16340001064,2024-03-10 07:01:17.034,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710053490000,1710054074000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1.710053e+12,correct,1710054074000,completed,1,,,A_1,RESPONSE,value
1,634,QAT,Qatar,16340021007,2024-03-11 08:41:51.535,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710143231000,1710146509000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1.710143e+12,correct,1710146509000,completed,1,,,A_1,RESPONSE,value
2,191,HRV,Croatia,11910095034,2024-03-12 23:53:40.479,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710227979000,1710229646000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1.710228e+12,correct,1710229646000,completed,1,,,A_1,RESPONSE,value
3,634,QAT,Qatar,16340048102,2024-03-13 20:46:10.313,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710225990000,1710227858000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1.710226e+12,correct,1710227858000,completed,1,,,A_1,RESPONSE,value
4,634,QAT,Qatar,16340050014,2024-03-13 20:46:25.613,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710229495000,1710230708000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1.710229e+12,correct,1710230708000,completed,1,,,A_1,RESPONSE,value


In [137]:
cbk = df6.loc[:,['qtiLabel','resp_cat']].drop_duplicates().assign(cbk_status='present')

In [138]:
dat_tao_qti = df6.qtiId.unique()
dat_cbk_qti = cbk.qtiLabel.unique()
list(set(dat_tao_qti) - set(dat_cbk_qti))

['FLAS303Q02',
 'FLARDGA1008',
 'FLALDGPREA1006',
 'FLALDTB1001',
 'FLARDGA1013',
 'FLARDGSA2003',
 'FLALDGPREA1008',
 'FLAS402Q02',
 'FLARDGSA1010',
 'FLAS301Q02',
 'FLARDGA1017',
 'FLARDGB1004',
 'FLAS402Q01',
 'FLAL2IMCC1001',
 'FLAS404Q02',
 'FLAS401Q03',
 'i651b3d23e03602394a50e8c772189ee02',
 'FLARDGA1010',
 'FLARDGA2012',
 'FLARDPGPREA1012',
 'FLAS301Q01',
 'FLAS102Q03',
 'FLARDGA1006',
 'FLAS104Q04',
 'FLAR5IMCB2001',
 'FLAS401Q01',
 'FLALDGA1010',
 'FLALDTB2003',
 'FLAS203Q02',
 'S_Part3_End',
 'FLARDGSA2014',
 'FLARDGA2006',
 'FLALDTB2011',
 'FLARGTSB1002',
 'FLARDGSA2012',
 'FLAR5IMCB1001',
 'FLAL2IMCC1002',
 'FLALDTB2008',
 'FLALDTB2004',
 'S_Part1_End',
 'FLALDTC1001',
 'FLARDGSA1009',
 'FLARDGA2004',
 'FLAS201Q02',
 'FLALDGPREA1001',
 'FLARGTSB1004',
 'FLARDGSA2001',
 'FLAS101Q07',
 'FLAS302Q03',
 'FLAS101Q04',
 'FLALDGA1004',
 'FLAS103Q05',
 'FLAS301Q03',
 'FLAS103Q06',
 'FLARDGB1011',
 'FLAS302Q01',
 'FLARDGA2009',
 'FLARDGB1002',
 'FLALDGA2011',
 'FLALDTB2005',
 'FLALD

In [139]:
list(set(dat_cbk_qti) - set(dat_tao_qti))

['cluster1-FLAS07-item-17',
 'cluster1-FLAL13-item-14',
 'cluster1-FLAR13-item-18',
 'cluster1-FLAL15-item-19',
 'cluster1-FLAR12-item-21',
 'cluster1-FLAR01-item-23',
 'cluster1-FLAR18-item-10',
 'cluster1-FLAR15-item-7',
 'cluster1-FLAR18-item-19',
 'cluster1-FLAL06-item-21',
 'cluster1-FLAL02-item-15',
 'cluster1-FLAR13-item-19',
 'cluster1-FLAL01-item-3',
 'cluster1-FLAR10-item-17',
 'cluster1-FLAR06-item-11',
 'cluster1-FLAS08-item-8',
 'cluster1-FLAR04-item-5',
 'cluster1-FLAR04-item-22',
 'cluster1-FLAL13-item-24',
 'cluster1-FLAR19-item-3',
 'cluster1-FLAR18-item-2',
 'cluster1-FLAS06-item-7',
 'cluster1-FLAR18-item-20',
 'cluster1-FLAL12-item-17',
 'cluster1-FLAR12-item-16',
 'cluster1-FLAS09-item-10',
 'cluster1-FLAR01-item-10',
 'cluster1-FLAR03-item-16',
 'cluster1-FLAR21-item-12',
 'cluster1-FLAR13-item-13',
 'cluster1-FLAR23-item-21',
 'cluster1-FLAL07-item-13',
 'cluster1-FLAR24-item-20',
 'cluster1-FLAL01-item-21',
 'cluster1-FLAS05-item-1',
 'cluster1-FLAR02-item-18',


In [140]:
import numpy as np

df7 = pd.merge(
    df6,
    cbk.assign(cbk_status = 'present'),
    how = 'left',
    on = ['qtiLabel','resp_cat']
)

df7['cbk_status'] = df7['cbk_status'].fillna('')
df7['in_cq'] = np.select([(df7['dup_std_item'] == '') & (df7['stim_item'] == '') & (df7['cbk_status'] != '')],'1',default = '0')
df7.head()

,isocntcd,isoalpha3,isoname,login,last_update_date,test_qti_id,sessionStartTime,sessionEndTime,qtiLabel,qtiId,...,submissionTime,completionStatus,count,dup_std_item,stim_item,resp_value,resp_cat,resp_type,cbk_status,in_cq
0,634,QAT,Qatar,16340001064,2024-03-10 07:01:17.034,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710053490000,1710054074000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1710054074000,completed,1,,,A_1,RESPONSE,value,present,1
1,634,QAT,Qatar,16340021007,2024-03-11 08:41:51.535,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710143231000,1710146509000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1710146509000,completed,1,,,A_1,RESPONSE,value,present,1
2,191,HRV,Croatia,11910095034,2024-03-12 23:53:40.479,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710227979000,1710229646000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1710229646000,completed,1,,,A_1,RESPONSE,value,present,1
3,634,QAT,Qatar,16340048102,2024-03-13 20:46:10.313,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710225990000,1710227858000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1710227858000,completed,1,,,A_1,RESPONSE,value,present,1
4,634,QAT,Qatar,16340050014,2024-03-13 20:46:25.613,FLA-R-L1-FLA-R-H1-FLA-R-M5,1710229495000,1710230708000,cluster1-FLAR22-item-1,FLARDPGPREA1001,...,1710230708000,completed,1,,,A_1,RESPONSE,value,present,1


In [22]:
df7.to_csv('./data/deliveryResults_FLA.csv',index = False)

In [24]:
# del_keys = list(del_df)
# raw_data = pd.DataFrame()
# raw_data_all = []
# for f in del_keys:
#   dat = get_dat(api_json_extract(del_df[f], entity = None, options = {'pandas': False}))
#   raw_data_all.append(dat)

# raw_data = pd.concat(raw_data_all,axis=0,ignore_index=True)

raw_data = all_deliveries_results
raw_data.head(5)

,metadata,sessionEndTime,metadataRaw,login,testQtiId,testQtiTitle,submissionTime,duration,score,deliveryId,...,sessionStartTime,maxScore,rawLtiParameters,testQtiLabel,rawItems,publicationTime,tenantId,items,_id,messageId
0,"{'PISA25 Domains': 'https://www.oecd.org/STQ',...",1706008452000,"[{'propertyLabel': 'Label', 'values': [{'vuri'...",QQ3,ZOVLEOIO,StQ,1706008452000,5,0,87040b6da779,...,1706008447000,0,"{'tenant_id': '18', 'sub': 'QQ3', 'https://pur...",StQ-development,"[{'numAttempts': 1, 'rawOutcomes': [{'identifi...",1706008275000,18,"{'ST410': {'numAttempts': 1, 'statusCorrect': ...",M1FRIzg3MDQwYjZkYTc3OSMwYTkyZmFiMzIzMDEzNGNjYT...,NaN
1,"{'PISA25 Domains': 'https://www.oecd.org/SCI',...",1706085753000,"[{'propertyLabel': 'Label', 'values': [{'vuri'...",CG-MNE20,HZTYIRAY,Science Coding Guide Items - Trend,1706085753000,307,0,a048b08baeed,...,1706085446000,0,"{'tenant_id': '18', 'sub': 'CG-MNE20', 'https:...",Science Coding Guide Items - Trend,"[{'numAttempts': 2, 'rawOutcomes': [{'identifi...",1705594156000,18,"{'item-9': {'numAttempts': 1, 'statusCorrect':...",MDJFTk0tR0MjYTA0OGIwOGJhZWVkIzRmODc1YzlmNGE4YW...,NaN
2,"{'PISA25 Domains': 'https://www.oecd.org/SCI',...",1706865971000,"[{'propertyLabel': 'Label', 'values': [{'vuri'...",CG-HKG20,HZTYIRAY,Science Coding Guide items - New,1706865971000,101958,2,7f5275e566c9,...,1706764013000,41,"{'tenant_id': '18', 'sub': 'CG-HKG20', 'https:...",Science Coding Guide Items - New,"[{'numAttempts': 1, 'rawOutcomes': [{'identifi...",1705589404000,18,"{'item-18': {'numAttempts': 1, 'statusCorrect'...",MDJHS0gtR0MjN2Y1Mjc1ZTU2NmM5IzJhNTgyMjMxMzFlNW...,NaN
3,"{'PISA25 Domains': 'https://www.oecd.org/SCI',...",1706756096000,"[{'propertyLabel': 'Label', 'values': [{'vuri'...",CG-LUX20,HZTYIRAY,Science Coding Guide Items - Trend,1706756096000,477,0,a048b08baeed,...,1706755619000,0,"{'tenant_id': '18', 'sub': 'CG-LUX20', 'https:...",Science Coding Guide Items - Trend,"[{'numAttempts': 2, 'rawOutcomes': [{'identifi...",1705594156000,18,"{'item-9': {'numAttempts': 1, 'statusCorrect':...",MDJYVUwtR0MjYTA0OGIwOGJhZWVkIzRmODc1YzlmNGE4YW...,NaN
4,"{'PISA25 Group': 'batch1', 'PISA25 Is Translat...",1707084391000,"[{'propertyLabel': 'Label', 'values': [{'vuri'...",TSC6030,CUSAS002,Fuel to Minimise Carbon Dioxide (CUSAS002),1707084391000,23,0,a0ccf30faf32,...,1707084368000,4,"{'sub': 'TSC6030', 'https://purl.imsglobal.org...",Fuel to Minimise Carbon Dioxide (CUSAS002),"[{'numAttempts': 1, 'rawOutcomes': [{'identifi...",1706479391000,18,"{'item-4': {'numAttempts': 1, 'statusCorrect':...",MDMwNkNTVCNhMGNjZjMwZmFmMzIjYWRhNjZlZGE3M2YzYT...,10342833531691882


In [69]:
import great_expectations as gx

raw_data = gx.from_pandas(raw_data)

In [70]:
def validateJSON(jsonData):
  try:
    json.loads(json.dumps(jsonData))
  except ValueError as err:
    return False
  return True

raw_data['valid_json'] = raw_data.apply(
  lambda d: validateJSON(d['items']),
  axis = 1
)

raw_data.head(5)
raw_data.expect_column_distinct_values_to_be_in_set(
  column = 'valid_json',
  value_set = [True]
)

{
  "success": true,
  "result": {
    "observed_value": [
      true
    ],
    "element_count": 1228,
    "missing_count": null,
    "missing_percent": null
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [34]:
def json_key_item(jsonData):
  try:
    if(isinstance(jsonData,list)):
      check_list = []
      for i in jsonData:
        keys = list(i.keys())
        check_key = all(('item-' in s) for s in keys)
        check_list.append(check_key)
      check=all(check_list)
    elif(isinstance(jsonData,dict)):
      keys = list(jsonData.keys())
      check = all(('item-' in s) for s in keys)
  except ValueError as err:
    return False
  return check

raw_data['item_keys'] = raw_data.apply(
  lambda d: json_key_item(d['items']),
  axis = 1
)

raw_data.head()
raw_data.expect_column_distinct_values_to_be_in_set(
  column = 'item_keys',
  value_set = [True]
)

{
  "success": false,
  "result": {
    "observed_value": [
      false,
      true
    ],
    "element_count": 1228,
    "missing_count": null,
    "missing_percent": null
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [ ]:
array = raw_data.items[0]
# keys = raw_data.items[0].keys()
# values = [raw_data.items[0][lab]['maxScore'] for lab in keys]
# values = [json.loads(raw_data.raw_data[0][lab]['responses']['RESPONSE']['value'])['ts'] for lab in keys]
# res = dict(map(lambda i,j : (i,j) , keys,values))
# l = [isinstance(s,str) for k,s in res.items()]
# all(l)

In [35]:
result_list = []
for k, v in raw_data.items.items():
  qti_label = list(pd.json_normalize(raw_data.raw_data[0],max_level=0).transpose().apply(
    lambda df: df[0]['qtiLabel']
  ))
  result_list.append(qti_label)

AttributeError: 'function' object has no attribute 'items'

In [72]:
from schemadict import schemadict, STANDARD_VALIDATORS

my_validators = STANDARD_VALIDATORS

my_validators['$required_keys'] = [
  'numAttempts',
  'statusCorrect',
  'qtiTitle',
  'maxScore',
  'submissionTime',
  'qtiId',
  'qtiLabel',
  'duration',
  'score',
  'itemEndTime',
  'outcomes',
  'responses',
  'completionStatus',
  'itemStartTime'
]

schema_outcome = schemadict({
  # "SCORE": {"type": str},
  "completionStatus": {"type": str}
})

schema_response = schemadict({
  "correct": {"type": bool},
  "value": {"type": str}
})

schema = schemadict(
  {
    "numAttempts": {"type": int, '>=': 0},
    "statusCorrect": {"type": str},
    'qtiTitle': {"type": str},
    # 'maxScore': {"type": str},
    'submissionTime': {"type": int},
    'qtiId': {"type": str},
    'qtiLabel': {"type": str},
    'duration': {"type": int},
    # 'score': {"type": int},
    'itemEndTime': {"type": int},
    'outcomes': {
      "type": dict,
      'item_type': dict,
      'item_schemadict': schema_outcome
    },
    'responses': {
      "type": dict,
      'item_type': dict,
      'item_schemadict': schema_response
    },
    'completionStatus': {"type": str},
    'itemStartTime' : {"type": int}
  },
  validators=my_validators
)

schema.keys()


KeysView(schemadict({'numAttempts': {'type': <class 'int'>, '>=': 0}, 'statusCorrect': {'type': <class 'str'>}, 'qtiTitle': {'type': <class 'str'>}, 'submissionTime': {'type': <class 'int'>}, 'qtiId': {'type': <class 'str'>}, 'qtiLabel': {'type': <class 'str'>}, 'duration': {'type': <class 'int'>}, 'itemEndTime': {'type': <class 'int'>}, 'outcomes': {'type': <class 'dict'>, 'item_type': <class 'dict'>, 'item_schemadict': schemadict({'completionStatus': {'type': <class 'str'>}})}, 'responses': {'type': <class 'dict'>, 'item_type': <class 'dict'>, 'item_schemadict': schemadict({'correct': {'type': <class 'bool'>}, 'value': {'type': <class 'str'>}})}, 'completionStatus': {'type': <class 'str'>}, 'itemStartTime': {'type': <class 'int'>}}))

In [90]:

# Describe what kind of json you expect.
raw_resp_schema = {
  "type": "object",
  "properties": {
    "numAttempts": {"type": "number"},
    "statusCorrect": {"type": "string"},
    'qtiTitle': {"type": "string"},
    # 'maxScore': {"type": "number"},
    'submissionTime': {"type": "number"},
    'qtiId': {"type": "str"},
    'qtiLabel': {"type": "str"},
    'duration': {"type": "number"},
    # 'score': {"type": "number"},
    'itemEndTime': {"type": "number"},
    'outcomes': {
      "type": "object",
      "properties": {
        #  "SCORE": {"type": "number"},
         "completionStatus": {"type": "string"}
      },
      "required": ['SCORE','completionStatus']
    },
    'responses': {
      "type": "object",
      "properties": {
        "RESPONSE": {
          "type": "object",
          "properties":{
             "correct": {"type": "boolean"},
            #  "value": {"type": "string"}
          },
          "required": ['correct','value']
        },
        "completionStatus": {"type": "string"},
        "itemStartTime": {"type": "number"}
      }
    },
    'completionStatus': {"type": "string"},
    'itemStartTime' : {"type": "number"},
  },
  "required": ['numAttempts',
    'statusCorrect',
    'qtiTitle',
    'maxScore',
    'submissionTime',
    'qtiId',
    'qtiLabel',
    'duration',
    'score',
    'itemEndTime',
    'outcomes',
    'responses',
    'completionStatus',
    'itemStartTime'
  ]
}


In [73]:
import jsonschema
from jsonschema import validate

def validateJsonSchema(jsonData,schema):
    try:
      if(isinstance(jsonData,list)):
        all([schema.validate(v) for v in jsonData])
      elif(isinstance(jsonData,dict)):
        all([schema.validate(v) for k,v in jsonData.items()])
    except TypeError as err:
      return str(err)
    return "Valid"

raw_data['item_json_schema'] = raw_data.apply(
  lambda d: validateJsonSchema(d['items'],schema),
  axis = 1
)

raw_data.head(5)
raw_data.expect_column_distinct_values_to_be_in_set(
  column = 'item_json_schema',
  value_set = ['Valid']
)

{
  "success": true,
  "result": {
    "observed_value": [
      "Valid"
    ],
    "element_count": 1228,
    "missing_count": null,
    "missing_percent": null
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [77]:
outcome = schema.validate(list(raw_data['items'][0].values())[0])
print(outcome)

None


In [ ]:
array = raw_data.raw_data[0].values()
keys = list(raw_data.raw_data[0].keys())
values = [raw_data.raw_data[0][lab]['responses'] for lab in keys]
# values = [json.loads(raw_data.raw_data[0][lab]['responses']) for lab in keys]
res = dict(map(lambda i,j : (i,j) , keys,values))
# l = [isinstance(s,str) for k,s in res.items()]
# all(l)

In [ ]:
print("Valid JSON: " + str(all(raw_data.valid_json)))
print("Items as keys: " + str(all(raw_data.item_keys)))
print("Correct JSON Schema: " + str(all(raw_data.item_json_schema)))

In [99]:
df = raw_data.explode('items').reset_index(drop = True)

In [100]:
df.head()

,metadata,sessionEndTime,metadataRaw,login,testQtiId,testQtiTitle,submissionTime,duration,score,deliveryId,...,testQtiLabel,rawItems,publicationTime,tenantId,items,_id,messageId,valid_json,item_keys,item_json_schema
0,"{'PISA25 Domains': 'https://www.oecd.org/STQ',...",1706008452000,"[{'propertyLabel': 'Label', 'values': [{'vuri'...",QQ3,ZOVLEOIO,StQ,1706008452000,5,0,87040b6da779,...,StQ-development,"[{'numAttempts': 1, 'rawOutcomes': [{'identifi...",1706008275000,18,ST410,M1FRIzg3MDQwYjZkYTc3OSMwYTkyZmFiMzIzMDEzNGNjYT...,NaN,True,False,Valid
1,"{'PISA25 Domains': 'https://www.oecd.org/STQ',...",1706008452000,"[{'propertyLabel': 'Label', 'values': [{'vuri'...",QQ3,ZOVLEOIO,StQ,1706008452000,5,0,87040b6da779,...,StQ-development,"[{'numAttempts': 1, 'rawOutcomes': [{'identifi...",1706008275000,18,ST411,M1FRIzg3MDQwYjZkYTc3OSMwYTkyZmFiMzIzMDEzNGNjYT...,NaN,True,False,Valid
2,"{'PISA25 Domains': 'https://www.oecd.org/SCI',...",1706085753000,"[{'propertyLabel': 'Label', 'values': [{'vuri'...",CG-MNE20,HZTYIRAY,Science Coding Guide Items - Trend,1706085753000,307,0,a048b08baeed,...,Science Coding Guide Items - Trend,"[{'numAttempts': 2, 'rawOutcomes': [{'identifi...",1705594156000,18,item-9,MDJFTk0tR0MjYTA0OGIwOGJhZWVkIzRmODc1YzlmNGE4YW...,NaN,True,True,Valid
3,"{'PISA25 Domains': 'https://www.oecd.org/SCI',...",1706085753000,"[{'propertyLabel': 'Label', 'values': [{'vuri'...",CG-MNE20,HZTYIRAY,Science Coding Guide Items - Trend,1706085753000,307,0,a048b08baeed,...,Science Coding Guide Items - Trend,"[{'numAttempts': 2, 'rawOutcomes': [{'identifi...",1705594156000,18,item-18,MDJFTk0tR0MjYTA0OGIwOGJhZWVkIzRmODc1YzlmNGE4YW...,NaN,True,True,Valid
4,"{'PISA25 Domains': 'https://www.oecd.org/SCI',...",1706085753000,"[{'propertyLabel': 'Label', 'values': [{'vuri'...",CG-MNE20,HZTYIRAY,Science Coding Guide Items - Trend,1706085753000,307,0,a048b08baeed,...,Science Coding Guide Items - Trend,"[{'numAttempts': 2, 'rawOutcomes': [{'identifi...",1705594156000,18,item-8,MDJFTk0tR0MjYTA0OGIwOGJhZWVkIzRmODc1YzlmNGE4YW...,NaN,True,True,Valid


In [95]:
tmp = (
  raw_data
  .join(
    pd.json_normalize(
      raw_data['items'],
      max_level = 0
    )
  )
  .drop(
    ['items'],
    axis = 1
  )
)

stub_cols = tmp.columns[tmp.columns.str.startswith("item-")]
id_cols = set(tmp.columns) - set(stub_cols)

raw_data_melt = tmp.melt(
  var_name = "items",
  value_name= "vars",
  id_vars = id_cols,
  value_vars = stub_cols
)

raw_data_melt.head(5)

,ST490,cluster2-CACERL009-item-12,ST804QCY,cluster1-CACERL002-item-9,cluster2-CACERL007-item-11,cluster2-CACERL008-item-3,cluster2-CACERL008-item-11,TC146FL,TC161FL,ST801FLDEU,...,maxScore,cluster2-CACERL007-item-9,cluster2-CPERS102-item-4,TC114FL,last_update_date,cluster2-S428-item1,cluster1-CACERL006-item-14,tenantId,items,vars
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,1706008455379,NaN,NaN,18,item-9,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,1706085756771,NaN,NaN,18,item-9,"{'numAttempts': 1, 'statusCorrect': 'incorrect..."
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,41,NaN,NaN,NaN,1706865974943,NaN,NaN,18,item-9,"{'numAttempts': 1, 'statusCorrect': 'correct',..."
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,1706756098586,NaN,NaN,18,item-9,"{'numAttempts': 1, 'statusCorrect': 'incorrect..."
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4,NaN,NaN,NaN,1707084393799,NaN,NaN,18,item-9,NaN


In [120]:
meta_cols = ['deliveryExecutionId','deliveryId','login']

s = raw_data.set_index(meta_cols)['items']
out = pd.DataFrame(s.tolist(),index = s.index).reset_index()

# tmp.head(5)

# stub_cols = tmp.columns[tmp.columns.str.startswith("item-")]
stub_cols = set(tmp.columns) - set(meta_cols)

raw_data_melt = tmp.melt(
  var_name = "items",
  value_name= "vars",
  id_vars = meta_cols,
  value_vars = stub_cols
).dropna()

df = raw_data_melt['vars'].apply(pd.Series)

result = pd.concat([raw_data_melt,df],axis = 1).drop('vars',axis = 1)

result.head(5)

,deliveryExecutionId,deliveryId,login,items,numAttempts,statusCorrect,qtiTitle,maxScore,submissionTime,qtiId,...,spec_lti_claim_custom_deliverySettings_plugins_disableCommands_enabled,spec_lti_claim_custom_deliverySettings_plugins_preventScreenshot_enabled,spec_lti_claim_custom_deliverySettings_plugins_forceFullScreen_enabled,spec_lti_claim_custom_deliverySettings_plugins_pauseOnBlur_autoresume,spec_lti_claim_custom_deliverySettings_attemptId,spec_lti_claim_custom_portalLaunch,spec_lti_claim_custom_deliverySettings_plugins_pauseOnBlur_enabled,SCORE_TOTAL_MAX,SCORE_RATIO,SCORE_TOTAL
2,02GKH-GC#7f5275e566c9#2a58223131e5a64a6a646b29...,7f5275e566c9,CG-HKG20,item-80,1.0,correct,CPERS102Q04,0,1.706866e+12,i63a147ebd300c26079455ec3fed58aa170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,10EHC-GC#7f5275e566c9#2a58223131e5a64a6a646b29...,7f5275e566c9,CG-CHE01,item-80,1.0,correct,CPERS102Q04,0,1.706756e+12,i63a147ebd300c26079455ec3fed58aa170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,10MNV-GC#7f5275e566c9#2a58223131e5a64a6a646b29...,7f5275e566c9,CG-VNM01,item-80,1.0,correct,CPERS102Q04,0,1.706866e+12,i63a147ebd300c26079455ec3fed58aa170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,10NAC-GC#7f5275e566c9#2a58223131e5a64a6a646b29...,7f5275e566c9,CG-CAN01,item-80,1.0,correct,CPERS102Q04,0,1.706752e+12,i63a147ebd300c26079455ec3fed58aa170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,10NRB-GC#7f5275e566c9#2a58223131e5a64a6a646b29...,7f5275e566c9,CG-BRN01,item-80,1.0,correct,CPERS102Q04,0,1.706609e+12,i63a147ebd300c26079455ec3fed58aa170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
df.head(5)

,numAttempts,statusCorrect,qtiTitle,maxScore,submissionTime,qtiId,qtiLabel,duration,score,itemEndTime,...,spec_lti_claim_custom_deliverySettings_plugins_disableCommands_enabled,spec_lti_claim_custom_deliverySettings_plugins_preventScreenshot_enabled,spec_lti_claim_custom_deliverySettings_plugins_forceFullScreen_enabled,spec_lti_claim_custom_deliverySettings_plugins_pauseOnBlur_autoresume,spec_lti_claim_custom_deliverySettings_attemptId,spec_lti_claim_custom_portalLaunch,spec_lti_claim_custom_deliverySettings_plugins_pauseOnBlur_enabled,SCORE_TOTAL_MAX,SCORE_RATIO,SCORE_TOTAL
2,1.0,correct,CPERS102Q04,0,1.706866e+12,i63a147ebd300c26079455ec3fed58aa170,CPERS102Q04,1.0,0,1.706866e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,1.0,correct,CPERS102Q04,0,1.706756e+12,i63a147ebd300c26079455ec3fed58aa170,CPERS102Q04,162.0,0,1.706695e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,1.0,correct,CPERS102Q04,0,1.706866e+12,i63a147ebd300c26079455ec3fed58aa170,CPERS102Q04,1.0,0,1.706866e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,1.0,correct,CPERS102Q04,0,1.706752e+12,i63a147ebd300c26079455ec3fed58aa170,CPERS102Q04,11.0,0,1.706280e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,1.0,correct,CPERS102Q04,0,1.706609e+12,i63a147ebd300c26079455ec3fed58aa170,CPERS102Q04,19.0,0,1.706608e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
json_dat = pd.json_normalize(raw_data_melt.vars.tolist())

meta_cols = [
  # 'battery_id',
  'delivery_execution_id',
  'delivery_id',
  'isDeleted',
  'last_update_date',
  'login',
  'test_qti_id',
  'test_qti_label',
  'test_qti_title',
  'raw_data'
]

id_cols2 = set(raw_data_melt.columns) - set(['vars'])
extra_cols = ['items','vars','values']
final_cols = meta_cols
final_cols.extend(extra_cols)

dat_long = (
  raw_data_melt
  .join(
    json_dat
  )
  .drop(
    ['vars'],
    axis = 1
  )
  .melt(
    var_name = "vars",
    value_name= "values",
    id_vars = id_cols2,
    value_vars = json_dat.columns
  )
  .reindex(
    columns=final_cols
  )
)

dat_long.head(20)

In [ ]:
dat = dat_long[dat_long['vars'].str.startswith('responses.')]
dat = dat[dat['vars'].str.endswith('value')]
dat

In [ ]:
d = raw_data.raw_data[0]
def flatten_dict(nested_dict):
    res = {}
    if isinstance(nested_dict, dict):
        for k in nested_dict:
            flattened_dict = flatten_dict(nested_dict[k])
            for key, val in flattened_dict.items():
                key = list(key)
                key.insert(0, k)
                res[tuple(key)] = val
    else:
        res[()] = nested_dict
    return res


def nested_dict_to_df(values_dict):
    flat_dict = flatten_dict(values_dict)
    df = pd.DataFrame.from_dict(flat_dict, orient="index")
    df.index = pd.MultiIndex.from_tuples(df.index)
    df = df.unstack(level=-1)
    df.columns = df.columns.map("{0[1]}".format)
    return df

df = nested_dict_to_df(d)

In [ ]:
import datetime

timestamp = "1705282053574"
your_dt = datetime.datetime.fromtimestamp(int(timestamp)/1000)  # using the local timezone
print(your_dt.strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
filename = 'datastoreDeliveryResults_d60f257c8d73.json'
string = json.dumps(output)
json_data = json.loads(
  string
    .replace("\\t", "\\\\t")
    .replace("\\n", "\\\\n")
)

# with open(filename, 'w') as output_data:
#   output_data.write(
#     json.dumps(json_data, ensure_ascii=False)
#   )
  

In [ ]:
with open(filename) as json_data:
  data = json.load(json_data)
  df = pd.DataFrame(data['data'])
  meta_data = pd.DataFrame(df.metadata.values.tolist()).drop_duplicates()

meta_data


In [ ]:
def flatten_json(nested_json, exclude=['']):
    """Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
            exclude: Keys to exclude from output.
        Returns:
            The flattened json object if successful, None otherwise.
    """
    out = {}

    def flatten(x, name='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude: flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

In [ ]:
def flatten(data, new_data):
    """Recursive helper function.

    Args:
        data: nested dictionary.
        new_data: empty dictionary.

    Returns:
        Flattened dictionary.

    """
    for key, value in data.items():
        if isinstance(value, dict):
            flatten(value, new_data)
        if isinstance(value, str) or isinstance(value, int) or isinstance(value, list):
            new_data[key] = value
    return new_data

In [ ]:
from functools import reduce

def _explode(df, col):
    df = df.explode(col)

    if isinstance(df.iloc[0][col], list):
        df = _explode(df, col)
    elif isinstance(df.iloc[0][col], object):
        df_child = pd.json_normalize(df[col])
        # To prevent column name collision, add the parent column name as prefix.
        df_child.columns = [f'{col}.{x}' for x in df_child.columns]
        df = pd.concat([df.loc[:, ~df.columns.isin([col])].reset_index(drop=True), df_child], axis=1)
    
    return df

def full_explode_normalize(df):
    # Extract list columns 
    explode_cols = [x for x in df.columns if isinstance(df.iloc[0][x], list)]
    if len(explode_cols) < 1:
        return df
    
    # Explode and normalize the list
    df = reduce(_explode, explode_cols, df)

    return df

df_explode = full_explode_normalize(all_deliveries)
# df
# df_explode.columns
# df_explode.to_csv('df_explode.csv')

In [ ]:
df_explode.head(5)

### Sourcing Data from RDBMS tables

In [ ]:
# Read sqlite query results into a pandas DataFrame
with sqlite3.connect("movies.sqlite") as conn:
    df = pd.read_sql("SELECT * from movies", conn)
df.head()

# Sourcing data from Webpages

Please visit the url https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)

In [ ]:
# get data from url
df_html = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)',match = 'by country')
# Let's see how many tables are there with tage ' by county'
print(len(df_html)) # There are 4 tables
# Let's see the first table
df_html[0]